In [31]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l'
            ,'m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [3]:
num_dic = {c:i for i,c in enumerate(char_arr)}
print(num_dic)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [4]:
dic_len = len(num_dic)
n_step = 3
n_hidden = 128
n_input = n_class = dic_len

In [5]:
input1 = []
for n in 'wrd':
    print(n)
    input1.append(num_dic[n])
    pass
np.eye(dic_len)[input1]

w
r
d


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
seq_data = ['word','wood','deep','dive','cold','cool','load','keep','kiss']

In [7]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data:
        x = [num_dic[n] for n in seq[:-1]]
        y = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[x])
        target_batch.append(y)
        pass
    return input_batch,target_batch

In [8]:
X = tf.placeholder(tf.float32,[None,n_step,n_input])
Y = tf.placeholder(tf.int32,[None])

W1 = tf.Variable(tf.random_normal([n_hidden,n_class]))
b1 = tf.Variable(tf.random_normal([n_class]))

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs,stats = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)
outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]
print(outputs)
model = tf.matmul(outputs,W1)+b1

Tensor("strided_slice:0", shape=(?, 128), dtype=float32)


In [9]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
input_batch,target_batch = make_batch(seq_data)
for epoch in range(10):  
    _,l_cost = sess.run([optimizer,cost],feed_dict={X:input_batch,Y:target_batch})
    print(l_cost)
    pass


4.6172633
0.4217196
0.6168722
0.06879638
0.44700703
0.15313767
0.01297481
0.25350764
0.057678845
0.000836001


In [19]:
prediction = tf.cast(tf.argmax(model,1),tf.int32)
prediction_check = tf.equal(prediction,Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check,tf.float32))
input_batch,target_batch = make_batch(seq_data)

In [23]:
predict, result = sess.run([prediction,accuracy],feed_dict={X:input_batch,Y:target_batch})

In [24]:
print(predict,result)

[ 3  3 15  4  3 11  3 15 18] 1.0


In [30]:
last_word = sess.run(model,feed_dict={X:input_batch[0:1]})
print(np.argmax(last_word,1))

[3]


In [29]:
print(target_batch[0:1])

[3]
